# RAG(검색-증강 생성)
`Retrieval-Augmented Generation`
- LLM이 외부 데이터를 컨텍스트로 활용

### 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader('./practice_rag/가상 아이돌 이용자 연구.pdf')
docs = loader.load()

print('원본 pdf 장수: ', len(docs))

원본 pdf 장수:  14


In [5]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자 당 1 청크 / 50글자는 겹치게 나눈다
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

# for doc in split_docs:
#     print('\n')
#     print(doc)

print('분할 후 chunk 수: ', len(split_docs)) # 72

분할 후 chunk 수:  107


In [ ]:
# 3. 벡터 임베딩 + 4. 벤터스토어 저장

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
# vectorstore.similarity_search('미국 대통령 관련 문서 가져오기', k=4)

[Document(id='843de807-3ab5-41bb-982b-fff12641281a', metadata={'producer': 'NuriMedia Co., Ltd. - www.nurimedia.co.kr', 'creator': '', 'creationdate': '2024-11-05T19:38:35+09:00', 'source': './practice_rag/가상 아이돌 이용자 연구.pdf', 'file_path': './practice_rag/가상 아이돌 이용자 연구.pdf', 'total_pages': 14, 'format': 'PDF 1.4', 'title': 'DBPIA-NURIMEDIA', 'author': 'NuriMedia Contents Team', 'subject': '', 'keywords': '', 'moddate': '2024-11-05T19:38:35+09:00', 'trapped': '', 'modDate': "D:20241105193835+09'00'", 'creationDate': "D:20241105193835+09'00'", 'page': 5}, page_content='40세 사이의 서강대 메타버스전문대학원생 및 일반인을 대상\n으로 온라인 설문 조사를 실시하였다. 연구 대상자는 가상 아\n이돌 콘텐츠의 주요 이용자층으로 판단되어 연구 분석에 적\n합한 대상으로 선정되었다. 온라인 설문조사는 전문 업체에 \n의뢰하여 2024년 7월 18일부터 7월 27일까지 10일간 진행\n되었으며, 편의 표집 방법을 통해 모집된 참가자에게 설문 링\n크가 제공되었다. 설문지는 총 30개 문항으로 구성되어, 이용\n동기, 만족도, 지속사용의도 등을 포함한 다양한 항목을 측정\n하였다. 총 200개의 설문지를 회수하였으며, 이 중 중복 응\n답, 일관된 패턴의 불성실 응답 등 데이터 품질이 낮은 설문\n지 32개를 제외하였다. 이러한 제외 기준은 연구의 신뢰성과 \n데이터의 정확성을 높이기 위해 설정되었다. 최종적으로 분석\n에 사용된 

In [8]:
# 5. 최종 답변
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# 프롬프트 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

chain.invoke('해당 문서 요약해줘.')

'이 문서는 가상 아이돌 이용자들의 특성과 이용 동기, 만족도, 지속 사용 의도 등을 조사한 연구 결과를 다루고 있습니다. 설문조사를 통해 수집한 데이터를 분석하여, 가상 아이돌 콘텐츠가 이용자 경험에 미치는 영향과 성장 전략을 모색하는 내용을 포함합니다. 또한, 연구의 한계와 향후 방향성에 대해서도 논의하고 있습니다.'